# What kind of doggo is that? Classifying Dog Breeds using Convolutional Neural Networks

## Task 4: Model Pipeline

In this task, we will show how to build an efficient model input pipeline based on our preprocessing steps from the previous task.

In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
filenames = glob.glob('Data/*/*.jpg')

In [3]:
labels = [x.split('/')[1] for x in filenames]

In [4]:
breeds = np.unique(labels)
breeds_to_label = {breed: ix for ix, breed in enumerate(breeds)}
breeds_to_label

{'american_bulldog': 0,
 'boxer': 1,
 'english_cocker_spaniel': 2,
 'samoyed': 3,
 'scottish_terrier': 4,
 'staffordshire_bull_terrier': 5}

In [5]:
filenames_train, filenames_test , y_train, y_test = train_test_split(filenames,
                                                                    labels, test_size = 0.2,stratify = labels, random_state = 8)

In [9]:
table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys = tf.constant(list(breeds_to_label.keys())),
        values = tf.constant(list(breeds_to_label.values()))
    ),
    default_value = tf.constant(-1)
)
input_tensor = tf.constant(['samoyed'], dtype = tf.string)
table.lookup(input_tensor)

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([3], dtype=int32)>

In [10]:
def reshape_rescale_image(image, height=100, width=100):
    image = tf.cast(image, tf.float32)/255.
    image = tf.image.resize_with_pad(image, target_height=height, target_width=width)
    return image

def parse_image(file_path):
    label = tf.strings.split(file_path, os.sep)[-2]
    image = tf.io.decode_image(tf.io.read_file(file_path), channels=3)
    image.set_shape([None, None, 3])
    return reshape_rescale_image(image), table.lookup(label)

In [12]:
dataset_train =  tf.data.Dataset.list_files(filenames_train)
dataset_train = dataset_train.map(parse_image).shuffle(8).batch(32).prefetch(buffer_size = 100).cache()

dataset_val = tf.data.Dataset.list_files(filenames_test)
dataset_val = dataset_val.map(parse_image).batch(32).prefetch(buffer_size = 100)

In [14]:
num_classes = len(breeds)

model = tf.keras.models.Sequential( [
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.2),
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.Dropout(0.3),
])


In [16]:
model.compile(optimizer  = tf.keras.optimizers.Adam(learning_rate= 1e-4),
             loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
             metrics = ['accuracy'])